In [1]:
#  import packages

import pandas as pd
import numpy as np
import requests
import csv
import json
from pandas.io.json import json_normalize
import re
# scipy import for percentile of score
from scipy import stats
import time

In [2]:
# path to file of historical rainfall data upto current month
path = r'C:\Users\rj71b\geo-projects\wheatbelt_rainfall_analyser\data\external\wa_silo_weather_data.csv'

# path to silo data with BOM July observations concatenated
path_silo_bom_concat = r'C:\Users\rj71b\geo-projects\wheatbelt_rainfall_analyser\data\interim\silo_bom_july_concat.csv'

In [3]:
df = pd.read_csv(path_silo_bom_concat)

In [4]:
df.tail()

,station,year,month,rain
310063,12320,2019,3,63.6
310064,12320,2019,4,8.0
310065,12320,2019,5,2.2
310066,12320,2019,6,96.9
310067,12320,2019,7,35.8


In [17]:
# make string version of original column
df['date'] = df['date'].astype(str)

In [18]:
# make the new columns using string indexing
df['year'] = df['date'].str[0:4].astype('int64')
df['month'] = df['date'].str[4:6].astype('int64')

In [19]:
# get rid of the extra variable (if you want)
df.drop('date', axis=1, inplace=True)

In [5]:
df.tail()

,station,year,month,rain
310063,12320,2019,3,63.6
310064,12320,2019,4,8.0
310065,12320,2019,5,2.2
310066,12320,2019,6,96.9
310067,12320,2019,7,35.8


## Code functions to calculate rainfall percentiles for BOM Stations
refer to percentiles for individual stations nb

## Use to get percentiles for individual stations and use in QGIS Inverse Distance Weighting

In [10]:
def current_month(df, Month, Year):
    """fn: returns dataframe with stations that have rainfall data for the most recent month"""
    current_month = df.loc[(df['month'] == Month) & (df['year']== Year)]
    station_list = current_month['station'].tolist()
    df_current = df[df['station'].isin(station_list)]
    return df_current

In [11]:
#  filter df_concat dataframe so as to return stations with rainfall observations for most recent month
df_cm = current_month(df, 7, 2019)

In [6]:
def get_percentile1(df, Month):
    """fn: function calculates mean rainfall for individual station for
    specified Month. Returns the percentile of score for the most recent month"""
    # Filter df to get month we want to calcute pecentile of score
    month = df.loc[df['month'] == Month]
    # groupby to get mean rainfall across each SA2 region for specified month
    g = month.groupby(['station','year'])['rain']
    g_mean = g.mean()
    # reset index then groupby again to get value of most recent month. Thisis score to use in percentile of score
    g_reset = g_mean.reset_index()
    # gets last month average rainfall accross SA2 regions
    last = g_reset.groupby(['station']).nth(-1)
    # convert last month rainfall to array to use in percentile of score
    last_a = np.asarray(last['rain'])
    # convert each SA2 region rainfall observations into a list. This is then used in percentile of score function
    rain_list = g_reset.groupby('station')['rain'].apply(list)
    # slice list so as to remove most recent month rainfall reading from list    
    rain_list_for_percentile = [rain_list[i][:-1] for i in rain_list.index]
    percentiles = [stats.percentileofscore(rain_list_for_percentile[i], last_a[i]) for i in range(len(last_a))]
    # get list of SA2 Names     
    Station_Name = [name for name in last.index]
    dictionary = dict(zip(Station_Name, percentiles))
    return dictionary

In [7]:
#  call get_percentile function. Returns dict with percentiles for month of July
dict_percentile = get_percentile1(df, 7)

In [8]:
len(dict_percentile)

198

In [9]:
dict_percentile

{8002: 30.76923076923077,
 8005: 8.461538461538462,
 8008: 27.692307692307693,
 8013: 15.384615384615385,
 8016: 23.846153846153847,
 8025: 20.0,
 8028: 20.0,
 8037: 26.153846153846153,
 8044: 4.615384615384615,
 8050: 6.923076923076923,
 8052: 16.923076923076923,
 8060: 40.76923076923077,
 8066: 24.615384615384617,
 8072: 33.84615384615385,
 8077: 46.92307692307692,
 8079: 46.15384615384615,
 8087: 43.84615384615385,
 8088: 0.0,
 8095: 10.76923076923077,
 8100: 16.153846153846153,
 8107: 31.53846153846154,
 8113: 10.76923076923077,
 8121: 26.923076923076923,
 8130: 16.153846153846153,
 8137: 46.92307692307692,
 8139: 33.84615384615385,
 8143: 13.076923076923077,
 8147: 25.384615384615383,
 8157: 19.23076923076923,
 8168: 12.307692307692308,
 8200: 40.76923076923077,
 8240: 43.84615384615385,
 8251: 6.923076923076923,
 8254: 40.76923076923077,
 8273: 31.53846153846154,
 8294: 12.307692307692308,
 8296: 23.076923076923077,
 8297: 26.153846153846153,
 9014: 9.23076923076923,
 9018: 16.92

In [10]:
# convert percentile dict into pandas dataframe
df_percentile = pd.DataFrame(list(dict_percentile.items()), columns=['station', 'percentile'])

In [11]:
df_percentile.head()

,station,percentile
0,8002,30.769231
1,8005,8.461538
2,8008,27.692308
3,8013,15.384615
4,8016,23.846154


## Read in station metadata.
### Will merge with percentile dataframe to allow for mapping on QGIS

In [12]:
path_meta = r'C:\Users\rj71b\geo-projects\wheatbelt_rainfall_analyser\data\external\bom_station_metadata\wa_station_metadata.csv'
df_meta = pd.read_csv(path_meta, usecols = [0,1,2,3])

In [13]:
df_meta.head()

,station,name,lat,lon
0,9971,ACTON PARK,-33.7845,115.4072
1,9804,ADINA,-33.8811,122.2167
2,9999,ALBANY AIRPORT,-34.9411,117.8158
3,9556,ALDERVALE,-33.9781,116.3336
4,10052,ALEPPO,-31.4453,117.2219


## Merge df_percentile and df_meta

In [14]:
df_merge = pd.merge(df_meta, df_percentile, on = 'station')

In [15]:
df_merge.head()

,station,name,lat,lon,percentile
0,9804,ADINA,-33.8811,122.2167,16.923077
1,9556,ALDERVALE,-33.9781,116.3336,6.153846
2,10894,ALLAN ROCKS,-32.6322,119.0958,26.923077
3,10502,AMELUP,-34.2553,118.2214,34.615385
4,10000,AMERY ACRES,-31.1683,117.0736,46.153846


In [19]:
df_merge.dtypes, df_merge.shape

(station         int64
 name           object
 lat           float64
 lon           float64
 percentile    float64
 dtype: object, (199, 5))

### df_merge percentiles were calculated using BOM july rainfall concatenated to SILO data
### Results now seem more inline with BOM July Deciles

In [18]:
# send to data/processed folder
df_merge.to_csv(r'C:\Users\rj71b\geo-projects\wheatbelt_rainfall_analyser\data\processed\201907_percentiles.csv', index = False)


In [34]:
# SILO interpolated data for July is showing too high rainfall for central wheatbelt compared to BOM monthly July data
# have filtered based on BOM observations as found in monthly rainfall list 
open_stations = [8002, 8005, 8008, 8013, 8016, 8025, 8028, 8037, 8044, 8050, 8052, 8060, 8066, 8072, 8077, 8079, 8087, 8088, 8095, 8100, 8107, 8113, 8121, 8130, 8137, 8139, 8143, 8147, 8157, 8168, 8200, 8240, 8251, 8254, 8273, 8294, 8296, 8297, 9014, 9018, 9033, 9037, 9040, 9114, 9131, 9178, 9210, 9515, 9519, 9538, 9542, 9552, 9556, 9573, 9579, 9581, 9585, 9587, 9590, 9592, 9599, 9603, 9607, 9617, 9619, 9626, 9628, 9631, 9633, 9635, 9654, 9661, 9738, 9739, 9752, 9754, 9769, 9772, 9789, 9803, 9804, 9805, 9822, 9842, 9848, 9877, 9930, 9961, 9964, 9968, 9994, 10000, 10007, 10009, 10011, 10016, 10019, 10026, 10032, 10034, 10040, 10041, 10044, 10055, 10058, 10061, 10073, 10076, 10077, 10092, 10097, 10102, 10104, 10111, 10112, 10121, 10122, 10124, 10126, 10134, 10135, 10136, 10140, 10143, 10145, 10149, 10151, 10152, 10155, 10158, 10192, 10286, 10294, 10311, 10502, 10503, 10505, 10508, 10515, 10518, 10520, 10524, 10525, 10527, 10530, 10531, 10534, 10536, 10541, 10542, 10546, 10564, 10565, 10568, 10581, 10582, 10584, 10595, 10606, 10612, 10614, 10619, 10622, 10626, 10627, 10628, 10633, 10634, 10635, 10641, 10643, 10647, 10654, 10662, 10665, 10670, 10692, 10696, 10700, 10702, 10707, 10729, 10792, 10831, 10866, 10878, 10889, 10894, 10905, 10911, 10916, 10917, 11003, 11008, 11017, 11019, 11052, 12009, 12011, 12026, 12044, 12064, 12071, 12083, 12201, 12223, 12320]

In [35]:
df_filtered = df_merge[df_merge['station'].isin(open_stations)] #change strings in wa_stations to int


In [37]:
df_filtered

,station,name,lat,lon,percentile
0,9804,ADINA,-33.8811,122.2167,16.923077
1,9556,ALDERVALE,-33.9781,116.3336,6.153846
3,10894,ALLAN ROCKS,-32.6322,119.0958,32.307692
4,10502,AMELUP,-34.2553,118.2214,34.615385
5,10000,AMERY ACRES,-31.1683,117.0736,46.153846
6,10696,AMRISTA PARK,-32.6597,118.6319,67.692308
7,10503,ARDATH,-32.0504,118.0684,40.769231
8,8273,ARENA,-29.3586,115.4503,32.307692
9,10505,ARTHUR RIVER,-33.3392,117.0350,23.076923
13,10508,BADGEBUP,-33.6361,117.9278,35.384615


In [97]:
df_filtered.to_csv(r'C:\Users\rj71b\geo-projects\wheatbelt_rainfall_analyser\data\processed\201907_percentiles.csv', index = False)